<a href="https://colab.research.google.com/github/bikash119/bikash119/blob/main/learn_pytorch_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## The Goal : 
`python train.py --model MODEL_NAME --batch_size BATCH_SIZE --lr LEARNING_RATE --num_epochs NUM_EPOCHS`

Example `python train.py --model tinyvgg --batch_size 32 --lr 0.001 --num_epochs 10`

<pre>
deep_learning_classification_prj
  ├── src
  │   ├── data_setup.py
  │   ├── train.py
  │   ├── engine.py
  │   ├── utils.py
  │   ├── model.py
  ├── model
  │   ├── model_from_script.pth
  │   ├── model_from_cells.pth
  ├── data
  │   ├── root_data_folder
  │       ├── train
  │           ├── class_a
  │               ├── img_1.jpg|png
  │               ├── ....
  │               ├── img_n.jpg|png
  │           ├── class_b
  │           ├── class_c
  └───────├── test
              ├── class_a
              ├── class_b
              ├── class_c
  
</pre>


## Cell to download images from a github repo

In [1]:
import os
import pathlib
from pathlib import Path
import requests
import zipfile

## Setup data folder
root_path = Path("deep_learning_classification/data/")
img_path = root_path / "pizza_steak_sushi"

if img_path.is_dir():
  print(f'{img_path} directory exists.')
else:
  print(f'Creating directory {img_path}')
  img_path.mkdir(parents=True, exist_ok=True)

## Download zip file 
with open(root_path/'pizz_steak_sushi.zip', 'wb') as f:
  request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
  print(f'Downloading zip file...')
  f.write(request.content)

## unzip the zip file
with zipfile.ZipFile(root_path/'pizz_steak_sushi.zip', 'r' ) as zip_ref:
  print(f'Unzipping data...')
  zip_ref.extractall(img_path)

## Remove zip file
os.remove(root_path/'pizz_steak_sushi.zip')

Creating directory deep_learning_classification_prj/data/pizza_steak_sushi
Unzipping data...


## Create pytorch Datasets and DataLoaders

In [3]:
%%writefile deep_learning_classification/src/data_setup.py
"""
Contains functionality to create pytorch DataLoaders from image classification usecases
"""

import os
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from typing import Tuple, Dict, List

NUM_WORKERS = os.cpu_count()
def create_dataloader(train_dir:str,
                      test_dir: str,
                      train_transforms: transforms.Compose,
                      test_transforms: transforms.Compose,
                      batch_size: int,
                      num_workers: int) -> Tuple[DataLoader, DataLoader, List[str]]:
  """
   Creates train and test dataloaders

    Takes in training directory and testing directory and creates Pytorch Datasets which are then 
    used to create Pytorch DataLoaders

    Args:
      train_dr(str) : Folder containing images to be used for training the model
      test_dir(str) : Folder containing images to be used for testing the model
      train_transforms ( transforms.Compose) : Transformation to be applied on the images used for training
      test_transform ( transforms.Compose) : Transformation to be applied on the images used for testing
      batch_size(int) : Size of mini batch or Number of samples per batch in each DataLoaders
      num_workers (int) : Number of workers per DataLoader ( mostly equals to the number of CPU)

    Returns:
      A Tuple containing 
        train_dataloader ( torch.utils.data.DataLoader): A pytorch DataLoader for training dataset
        test_dataloader ( torch.utils.data.DataLoader) : A pytorch DataLoader for testing dataset
        classes ( List[str]) : A list of string representing the image classes.
    
    Example usage:
      train_dataloader, test_dataloader, classes = create_dataloaders(train_dir = path/to/train_img/folder,
                                                                      test_dir = path/to/train_img/folder,
                                                                      train_transforms= some_tranforms,
                                                                      test_transforms = some_transforms,
                                                                      batch_size= 32,
                                                                      num_workers = 2)
  """

  # Create pytorch datasets using images in train and test folder
  train_dataset = datasets.ImageFolder(root=train_dir,
                                       transform=train_transforms)
  test_dataset = datasets.ImageFolder(root=test_dir,
                                      transform=test_transforms)
  
  ## Get Image classes
  classes = train_dataset.classes

  # Create pytorch dataloaders from datasets
  train_dataloader = DataLoader(dataset=train_dataset,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=NUM_WORKERS)
  
  test_dataloader = DataLoader(dataset=test_dataset,
                               batch_size=batch_size,
                               shuffle=False,
                               num_workers=NUM_WORKERS)
  
  return train_dataloader, test_dataloader, classes
  

Overwriting deep_learning_classification/src/data_setup.py


In [11]:
import sys
sys.path.append('/content/deep_learning_classification')
from src import data_setup
from torchvision import transforms
train_dir = img_path/'train'
test_dir = img_path/'test'
transforms=transforms.Compose([
    transforms.Resize((64,64))
])
train_dataloader , test_dataloader, classes = create_dataloader(train_dir= train_dir,
                                                                test_dir = test_dir,
                                                                train_transforms=transforms,
                                                                test_transforms=transforms,
                                                                batch_size=32,
                                                                num_workers=NUM_WORKERS)

train_dataloader, test_dataloader, classes

(<torch.utils.data.dataloader.DataLoader at 0x7fabf8ec6b30>,
 ['pizza', 'steak', 'sushi'])

## Make the model

In [71]:
%%writefile deep_learning_classification/src/model.py
"""
  Contains the model architecture code
"""


import torch
from torch import nn

class TinyVGG(nn.Module):

  """
    Create the TinyVGG architecure

    Replicates the TinyVGG architecture from the CNN Explainer website in pytorch.
    See the original architecture here: https://poloclub.github.io

    Args:
      input_shape(int) : An integer indicating number of input channels.
      hidden_units(int) : An integer indicating number of hidden units between layers
      output_shape(int) : An integer indicating number of classes

    Example Usage
      model_v0 = TinyVGG(input_shape=3,
                         hidden_units=10,
                         output_units=3)
  """
  def __init__(self, 
               input_shape: int,
               hidden_units:int,
               output_shape: int) -> None:
    
    super().__init__()

    self.block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )

    self.block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

    self.classifier= nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units * 13 * 13,
                  out_features=output_shape)
    )

  def forward(self,x)-> torch.Tensor:
    return self.classifier(self.block_2(self.block_1(x)))


Overwriting deep_learning_classification/src/model.py


In [72]:
from src import model
import torch

torch.manual_seed(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

model_v2 = TinyVGG(input_shape=3, 
                   hidden_units=10, 
                   output_shape=len(classes)).to(device)

model_v2

TinyVGG(
  (block_1): Sequential(
    (0): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=1690, out_features=3, bias=True)
  )
)

In [73]:
try:
  import torchinfo
except:
  !pip install torchinfo
  import torchinfo

from torchinfo import summary
summary(model_v2,input_size=[1,3,64,64])

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [1, 3]                    --
├─Sequential: 1-1                        [1, 10, 30, 30]           --
│    └─Conv2d: 2-1                       [1, 10, 62, 62]           280
│    └─ReLU: 2-2                         [1, 10, 62, 62]           --
│    └─Conv2d: 2-3                       [1, 10, 60, 60]           910
│    └─ReLU: 2-4                         [1, 10, 60, 60]           --
│    └─MaxPool2d: 2-5                    [1, 10, 30, 30]           --
├─Sequential: 1-2                        [1, 10, 13, 13]           --
│    └─Conv2d: 2-6                       [1, 10, 28, 28]           910
│    └─ReLU: 2-7                         [1, 10, 28, 28]           --
│    └─Conv2d: 2-8                       [1, 10, 26, 26]           910
│    └─ReLU: 2-9                         [1, 10, 26, 26]           --
│    └─MaxPool2d: 2-10                   [1, 10, 13, 13]           --
├─Sequentia

## Create the Training engine.

In [3]:
%%writefile deep_learning_classification/src/engine.py
"""
  Contains function for training and testing a pytorch model
"""

import torch
from tqdm.auto import tqdm
from typing import Dict, List, Tuple


def train_step(model: nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device
               )-> Tuple[float, float]:
  """
    Trains a pytorch model for single Epoch

    Turns a target Pytorch model to training mode and then executes all the training steps
    forward pass,
    loss calculation,
    zero gradient,
    backpropagation
    update parameters
    Args:
      model(nn.Module): Model to be trained
      dataloader(torch.utils.data.DataLoader) : A DataLoader to be used for training the model
      loss_fn(nn.Module) : A function to calculate the loss
      optimizer(torch.optim.Optimzer) : A pytorch optimizer to help minimize the loss
      device(torch. device) : A target device to compute on 'cpu' or 'cuda'
    
    Returns:
      A tuple of training loss and training accuracy metrics in the form of
      (training_loss, training_acc)

    Example Usage : 
      train_loss, train_acc = train_step(model=model_v0, 
                                         dataloader=train_dataloader,
                                         loss_fn= nn.CrossEntropyLoss(),
                                         optimizer=torch.optim.Adam(params=model_v0.parameters(),lr=0.001),
                                         device='cuda'
      )
    
  """

  #Put the model to train mode
  model.train()

  #Setup the train loss and train accuracy
  train_loss, train_acc = 0,0

  # Iterate over the dataloader
  for batch,(X,y) in enumerate(dataloader):

    #send data to device
    X,y = X.to(device),y.to(device)

    #Forward pass
    pred_logits = model(X)

    # Calculate Loss
    loss = loss_fn(pred_logits, y)
    train_loss += loss.item()

    # Calculate Acc
    pred_label = torch.argmax(torch.softmax(pred_logits,dim=1),dim=1)
    train_acc += (pred_label == y).sum().item()/ len(pred_logits)

    # Zero gradient
    optimizer.zero_grad()

    # Back prop
    loss.backward()

    # update parameters
    optimizer.step()

  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  return train_loss, train_acc


def test_step(model: nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: nn.Module,
              device:torch.device)-> Tuple[float,float]:

    """
    Tests a pytorch model for single Epoch

    Turns a target Pytorch model to eval mode and then executes all the testing steps
    forward pass,
    loss calculation,
    
    Args:
      model(nn.Module): Model to be trained
      dataloader(torch.utils.data.DataLoader) : A DataLoader to be used for testing the model
      loss_fn(nn.Module) : A function to calculate the loss
      device(torch. device) : A target device to compute on 'cpu' or 'cuda'
    
    Returns:
      A tuple of testing loss and testing accuracy metrics in the form of
      (testing_loss, testing_acc)

    Example Usage : 
      test_loss, test_acc = test_step(model=model_v0, 
                                         dataloader=train_dataloader,
                                         loss_fn= nn.CrossEntropyLoss(),
                                         device='cuda'
      )
    
  """

    #Put the model in eval mode
    model.eval()

    # Intialize the test_loss and test_acc
    test_loss, test_acc = 0,0
    
    #Turn on the inference context manager
    with torch.inference_mode():
      
      # Loop over the dataloader
      for batch,(X,y) in enumerate(dataloader):
        
        # Send data to target device
        X,y = X.to(device),y.to(device)

        
        # Forward pass
        pred_logits = model(X)

        # Calculate Loss
        loss = loss_fn(pred_logits,y)
        test_loss += loss.item()
        
        # Calculate acc
        pred_labels = torch.argmax(torch.softmax(pred_logits,dim=1),dim=1)
        test_acc += (pred_labels == y).sum().item()/ len(pred_labels) 

      # Calculate the avg loss and accuracy across all batches.  
      test_loss /= len(dataloader)
      test_acc /= len(dataloader)
    return test_loss, test_acc


def train(model: nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: nn.Module,
          optimizer: torch.optim.Optimizer,
          device: torch.device,
          epochs: int) -> Dict[str,List[float]]
  
  """
    Trains and Tests the pytorch model

    Passes a pytorch model through the train_step and the test_step function for
    a epochs number of times.
    Args:
      model(nn.Module): Model to be trained
      train_dataloader(torch.utils.data.DataLoader) : A DataLoader to be used for training the model
      test_dataloader(torch.utils.data.DataLoader) : A DataLoader to be used for testing the model
      loss_fn(nn.Module) : A function to calculate the loss
      optimizer(torch.optim.Optimzer) : A pytorch optimizer to help minimize the loss
      device(torch.device) : A target device to compute on 'cpu' or 'cuda'
      epochs(int): Number of times the model should pass over the training and the testing dataset.
    
    Returns:
      A dictionary containing the train_loss, train_acc, test_loss & test_acc. 
      Each metrics have a value in list for each epoch

      The output form :
        {
          train_loss : List[float]
          train_acc : List[float]
          test_loss : List[float]
          test_acc : List[float]
        }
  
  """
  # Create a empty dictionary to hold the results

  results = {
      'train_loss':[]
      ,'train_acc':[]
      ,'test_loss':[]
      ,'test_acc':[]
  }

  

  for epoch in tqdm(range(epochs)):
    train_loss , train_acc = train_step(model=model,
                                        dataloader=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer,
                                        device=device)
    test_loss, test_acc = test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    optimizer=optimizer,
                                    device=device)

    print(f'Epoch : {epoch+1} |'
          f'train_loss : {train_loss:.4f} |'
          f'train_acc : {train_acc:.4f} |'
          f'test_loss : {test_loss:.4f} |'
          f'test_acc : {test_acc:.4f} |' 
    )
    results['train_loss'].append(train_loss)
    results['train_acc'].append(train_loss)
    results['test_loss'].append(train_loss)
    results['test_acc'].append(train_loss)
  
  return results





Writing deep_learning_classification/src/engine.py


In [4]:
%%writefile deep_learning_classification/src/utils.py
"""
  Contains various utility functions for pytorch model training and saving
"""

import torch
from torch import nn
from pathlib import Path

def save_mode(model: nn.Module,
              target_dir: str,
              model_name: str):
  """
    Saves a pytorch model to a target directory
    Args:
      model (nn.Module): a pytorch model to be saved
      target_dir (str): A directory for saving the model
      model_name (str): The filename to given to the model. Should include either 
                        .pt or .pth as the file extension 

    Returns : None

    Example usage
      save_model(model=model_0,
                 target_dir="models",
                 model_name="deep_learning_classification.pth")
  """

  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,exist_ok=True)

  # Create model save path
  assert model_name.endswith('.pt') or model_name.endswith('pth'),"model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path/model_name

  # Save the model state_dict()
  print(f'[INFO] Saving model to : {model_save_path}')
  torch.save(obj=model.state_dict(),
             f=model_save_path)
  



Writing deep_learning_classification/src/utils.py
